# 02 - Distributed Training with Feast + Remote KubeRay

![Workflow](../docs/02-training-workflow.png)

## What This Notebook Does

| Step | Component | Action |
|------|-----------|--------|
| 1 | Feast + Ray | `get_historical_features()` via remote KubeRay cluster |
| 2 | PyTorch | GPU-accelerated training |
| 3 | Artifacts | Save model to shared PVC |

## Architecture

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│     Feast       │────▶│    KubeRay      │────▶│   Training      │
│  FeatureStore   │     │  "feast-ray"    │     │  (PyTorch GPU)  │
└─────────────────┘     └─────────────────┘     └─────────────────┘
        │                       │                       │
        │ get_historical_       │ Distributed           │ GPU
        │ features()            │ PIT Joins             │ Training
        │                       │ (with mTLS)           │
```

**Recommended:** Use `kubectl apply -f manifests/06-trainjob.yaml` instead of this notebook for production.

**Prerequisites:** `01-feast-features.ipynb` completed, RayCluster `feast-ray` running.

In [ ]:
%pip install -q "feast[postgres,ray]==0.59.0" codeflare-sdk psycopg2-binary scikit-learn joblib pandas pyarrow torch

In [ ]:
import os
import json
import shutil
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
import joblib
from datetime import datetime, timezone, timedelta
from pathlib import Path

## Configuration

In [ ]:
# Paths
SHARED_ROOT = Path("/opt/app-root/src/shared")
FEATURE_REPO = SHARED_ROOT / "feature_repo"
DATA_DIR = SHARED_ROOT / "data"
OUTPUT_DIR = SHARED_ROOT / "models"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Training params
NUM_EPOCHS = 20
BATCH_SIZE = 512
LEARNING_RATE = 1e-3

# Ray/CodeFlare config
RAY_CLUSTER = "feast-ray"
NAMESPACE = "feast-trainer-demo"

print(f"📁 Feature repo: {FEATURE_REPO}")
print(f"📁 Output dir: {OUTPUT_DIR}")
print(f"🎯 Epochs: {NUM_EPOCHS}")

## Setup CodeFlare SDK Auth

Configure authentication for KubeRay cluster:

In [ ]:
# Setup CodeFlare SDK auth from service account token
token_path = "/var/run/secrets/kubernetes.io/serviceaccount/token"
if os.path.exists(token_path):
    with open(token_path) as f:
        os.environ["FEAST_RAY_AUTH_TOKEN"] = f.read().strip()
    k8s_host = os.environ.get("KUBERNETES_SERVICE_HOST")
    k8s_port = os.environ.get("KUBERNETES_SERVICE_PORT", "443")
    os.environ["FEAST_RAY_AUTH_SERVER"] = f"https://{k8s_host}:{k8s_port}"
    os.environ["FEAST_RAY_SKIP_TLS"] = "true"
    print("🔐 CodeFlare SDK auth configured from service account")
else:
    print("⚠️ No service account token found - using default auth")

# Set Ray cluster info
os.environ["FEAST_RAY_USE_KUBERAY"] = "true"
os.environ["FEAST_RAY_CLUSTER_NAME"] = RAY_CLUSTER
os.environ["FEAST_RAY_NAMESPACE"] = NAMESPACE

## Setup Feast with Ray Config

Use the Ray-enabled config for distributed `get_historical_features()`:

In [ ]:
# Copy Ray config as main config
ray_config = FEATURE_REPO / "feature_store_ray.yaml"
if ray_config.exists():
    shutil.copy(ray_config, FEATURE_REPO / "feature_store.yaml")
    print(f"✅ Using Ray config: {ray_config}")
else:
    print(f"⚠️ Ray config not found, using existing feature_store.yaml")

## Get Historical Features via Ray

This is the key Feast operation - retrieve features with point-in-time correctness:

```
Entity DataFrame (65K rows)
    │
    ▼
Feast get_historical_features()
    │
    ├── Connect to KubeRay via CodeFlare SDK
    ├── Distribute PIT joins across workers
    └── Return feature matrix
```

In [ ]:
from feast import FeatureStore

store = FeatureStore(repo_path=str(FEATURE_REPO))
print(f"✅ FeatureStore loaded")
print(f"   Feature services: {[fs.name for fs in store.list_feature_services()]}")

In [ ]:
# Create entity DataFrame (what we want features for)
print("Creating entity DataFrame...")

entity_rows = []
base_date = datetime(2022, 1, 1, tzinfo=timezone.utc)
NUM_WEEKS, NUM_STORES, NUM_DEPTS = 104, 45, 14

for week in range(NUM_WEEKS):
    event_ts = base_date + timedelta(weeks=week)
    for store_id in range(1, NUM_STORES + 1):
        for dept_id in range(1, NUM_DEPTS + 1):
            entity_rows.append({
                "store_id": store_id,
                "dept_id": dept_id,
                "event_timestamp": event_ts
            })

entity_df = pd.DataFrame(entity_rows)
print(f"✅ Entity DataFrame: {len(entity_df):,} rows")
print(f"   ({NUM_WEEKS} weeks × {NUM_STORES} stores × {NUM_DEPTS} depts)")

In [ ]:
# Get historical features via Ray
print("\n🚀 Calling get_historical_features() via KubeRay...")
print("   This distributes PIT joins across the Ray cluster")
print("   Expected time: 2-5 minutes for 65K rows")
print()

start_time = time.time()

training_data = store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service("training_features"),
)
training_df = training_data.to_df()

elapsed = time.time() - start_time
throughput = len(entity_df) / elapsed if elapsed > 0 else 0

print(f"\n✅ Features retrieved!")
print(f"   Rows: {len(training_df):,}")
print(f"   Columns: {len(training_df.columns)}")
print(f"   Time: {elapsed:.1f}s")
print(f"   Throughput: {throughput:,.0f} rows/sec")

In [ ]:
# 📊 SAMPLE DATA: Retrieved features from KubeRay
print("📊 Sample: Retrieved features via Remote KubeRay")
print(f"   Total columns: {len(training_df.columns)}")
print(f"   Feature columns: {[c for c in training_df.columns if c not in ['store_id', 'dept_id', 'event_timestamp']]}")
print()
training_df.head(10)

## Prepare Training Data

In [ ]:
# Drop rows with missing target
training_df = training_df.dropna(subset=["weekly_sales"])
print(f"Rows after dropping NaN: {len(training_df):,}")

# Temporal split (80/20)
training_df = training_df.sort_values("event_timestamp")
split_idx = int(len(training_df) * 0.8)
train_df = training_df.iloc[:split_idx]
val_df = training_df.iloc[split_idx:]

print(f"Train: {len(train_df):,} rows ({train_df['event_timestamp'].min().date()} to {train_df['event_timestamp'].max().date()})")
print(f"Val: {len(val_df):,} rows ({val_df['event_timestamp'].min().date()} to {val_df['event_timestamp'].max().date()})")

In [ ]:
# Feature columns (exclude identifiers and target)
exclude_cols = ["store_id", "dept_id", "date", "event_timestamp", "weekly_sales"]
feature_cols = [
    c for c in training_df.columns 
    if c not in exclude_cols
    and training_df[c].dtype in [np.float64, np.int64, np.float32, np.int32]
]

print(f"Feature columns ({len(feature_cols)}): {feature_cols}")

In [ ]:
# Prepare arrays
X_train = train_df[feature_cols].fillna(0).values
y_train = train_df["weekly_sales"].values
X_val = val_df[feature_cols].fillna(0).values
y_val = val_df["weekly_sales"].values

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Log transform + scale target (better for % error optimization)
y_scaler = StandardScaler()
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)
y_train_scaled = y_scaler.fit_transform(y_train_log.reshape(-1, 1)).flatten()
y_val_scaled = y_scaler.transform(y_val_log.reshape(-1, 1)).flatten()

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train_scaled.shape}")

In [ ]:
# Save scalers
joblib.dump(scaler, OUTPUT_DIR / "scaler.pkl")
joblib.dump(y_scaler, OUTPUT_DIR / "y_scaler.pkl")
joblib.dump(feature_cols, OUTPUT_DIR / "feature_cols.pkl")
joblib.dump({
    "scaler_X": scaler,
    "scaler_y": y_scaler,
    "use_log_transform": True
}, OUTPUT_DIR / "scalers.joblib")
print(f"✅ Scalers saved to {OUTPUT_DIR}")

## Model Definition

In [ ]:
class SalesMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], dropout=0.2):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, dim),
                nn.BatchNorm1d(dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = dim
        layers.append(nn.Linear(prev_dim, 1))
        self.net = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.net(x).squeeze(-1)


class SalesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self): 
        return len(self.X)
    def __getitem__(self, i): 
        return self.X[i], self.y[i]

## Training

In [ ]:
# Create dataloaders
train_dataset = SalesDataset(X_train, y_train_scaled)
val_dataset = SalesDataset(X_val, y_val_scaled)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

In [ ]:
# Initialize model
input_dim = X_train.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SalesMLP(input_dim).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
criterion = nn.MSELoss()

print(f"Device: {device}")
print(f"Model params: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Training loop
best_val_loss = float('inf')
history = []

print(f"\n🚀 Training for {NUM_EPOCHS} epochs...")
print("=" * 60)

for epoch in range(NUM_EPOCHS):
    # Train
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        pred = model(X_batch)
        loss = criterion(pred, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    
    # Validate
    model.eval()
    val_loss = 0
    val_preds = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            pred = model(X_batch)
            val_loss += criterion(pred, y_batch).item()
            val_preds.extend(pred.cpu().numpy())
    val_loss /= len(val_loader)
    
    # Calculate MAPE
    val_preds = np.array(val_preds)
    pred_orig = np.expm1(y_scaler.inverse_transform(val_preds.reshape(-1, 1)).flatten())
    mask = y_val > 1000
    mape = np.mean(np.abs((y_val[mask] - pred_orig[mask]) / y_val[mask])) * 100
    
    scheduler.step()
    history.append({"epoch": epoch + 1, "train_loss": train_loss, "val_loss": val_loss, "mape": mape})
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), OUTPUT_DIR / "model_best.pt")
    
    # Print progress
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1:3d}/{NUM_EPOCHS} | Train: {train_loss:.4f} | Val: {val_loss:.4f} | MAPE: {mape:.1f}%")

In [ ]:
# Save final model
torch.save(model.state_dict(), OUTPUT_DIR / "model_final.pt")

# Save model metadata
best_mape = min(h["mape"] for h in history)
model_info = {
    "model_type": "SalesMLP",
    "input_dim": input_dim,
    "hidden_dims": [256, 128, 64],
    "dropout": 0.2,
    "best_val_loss": best_val_loss,
    "best_mape": best_mape,
    "epochs": NUM_EPOCHS,
    "feature_columns": feature_cols,
    "timestamp": datetime.now(timezone.utc).isoformat()
}

with open(OUTPUT_DIR / "model_metadata.json", "w") as f:
    json.dump(model_info, f, indent=2)

print(f"\n" + "=" * 60)
print(f"✅ TRAINING COMPLETE")
print(f"=" * 60)
print(f"Best Val Loss: {best_val_loss:.4f}")
print(f"Best MAPE: {best_mape:.1f}%")
print(f"\nArtifacts saved to: {OUTPUT_DIR}")
print(f"  - model_best.pt")
print(f"  - model_final.pt")
print(f"  - scalers.joblib")
print(f"  - model_metadata.json")

## Training History

In [ ]:
history_df = pd.DataFrame(history)
history_df

## ✅ Complete!

### Artifacts

| File | Purpose |
|------|----------|
| `model_best.pt` | Best model weights |
| `model_final.pt` | Final model weights |
| `scalers.joblib` | Feature and target scalers |
| `model_metadata.json` | Architecture info, feature columns |

### Alternative: Use TrainJob Manifest

For production distributed training:

```bash
kubectl apply -f manifests/06-trainjob.yaml
kubectl logs -f -l app.kubernetes.io/name=sales-forecasting -n feast-trainer-demo
```

### Next Steps

- `03-inference.ipynb` → Deploy model with KServe